# 1. Import and Option

In [1]:
import pandas as pd
import requests
import json

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 15)

# 2. Get api key
[Check Open Weather Map API key](https://home.openweathermap.org/api_keys)

In [2]:
# API
key = input('openweathermap에서 복사한 api key를 입력해주세요 > ')

openweathermap에서 복사한 api key를 입력해주세요 > a


# 3. Choose cities to check weather
[도시 ID 정보 : city.list.json.gz](http://bulk.openweathermap.org/sample/)

In [51]:
# 1. 날씨 확인할 도시 직접 지정
cities = [
    "Seoul,KR",
    "Tokyo,JP",
    "New York,US",
    "Tonghae,KR",
    "Seongnam-si,KR",
    "Hanam,KR",
    "Gyeongsangnam-do,KR",
]

In [57]:
# 2. 국내 모든 도시 지정 ()
import gzip
import json

with gzip.open('city.list.json.gz', 'r') as f:
    content = f.read()
    
json_str = content.decode('utf-8')
data = json.loads(json_str)

df_city = pd.DataFrame(data)
df_kr = df_city[df_city['country']=='KR']

# Dataframe의 두 문자열 Series의 원소 합치기
# https://rfriend.tistory.com/389
df_kr_city = df_kr[['name', 'country']].apply(lambda x: ','.join(x), axis=1)

cities_list = df_kr_city.to_list()
print(f"국내 날씨 정보를 불러올 수 있는 도시 수 : {len(cities_list)}개")

국내 날씨 정보를 불러올 수 있는 도시 수 : 242개


In [58]:
cities_list

['Tokusan-ri,KR',
 'Heunghae,KR',
 'Reisui,KR',
 'Yeonil,KR',
 'Neietsu,KR',
 'Eisen,KR',
 'Yongin,KR',
 'Yeonggwang,KR',
 'Yeongdong,KR',
 'Eisen,KR',
 'Yeongam-guncheong,KR',
 'Yeoncheon-gun,KR',
 'Yeoju,KR',
 'Yesan,KR',
 'Yecheon,KR',
 'Yangyang,KR',
 'Yangsan,KR',
 "Yangp'yŏng,KR",
 'Yangju,KR',
 'Yanggu,KR',
 'Yach’on,KR',
 'Wŏnju,KR',
 'Wanju,KR',
 'Waegwan,KR',
 'Unsal-li,KR',
 'Umulmok,KR',
 'Ulsan,KR',
 'Ulsan,KR',
 'Ulchin,KR',
 'Uijeongbu-si,KR',
 'Tangjin,KR',
 'Taesal-li,KR',
 'Daejeon,KR',
 'Daejeon,KR',
 'Daegu,KR',
 'Daegu,KR',
 'Boryeong,KR',
 'T’aebaek,KR',
 'Taian,KR',
 'Suwon,KR',
 'Suncheon,KR',
 'Sunchang-chodeunghakgyo,KR',
 'Republic of Korea,KR',
 'Seoul,KR',
 'Seoul,KR',
 'Seosan,KR',
 'Jenzan,KR',
 'Seisan-ri,KR',
 'Songjeong,KR',
 'Seonghwan,KR',
 'Sokcho,KR',
 'Yongsan,KR',
 'Sinch’ŏn-dong,KR',
 'Sangok,KR',
 'Sang-ni,KR',
 'Sangju,KR',
 'Santyoku,KR',
 'Shisen,KR',
 'Pyeongtaek,KR',
 'Pyeong,KR',
 'Buyeo,KR',
 'Busan,KR',
 'Busan,KR',
 'Bucheon-si,KR',
 '

# 4. Get Dataframe from url

In [61]:
urls = []

# 도시 id 리스트 : cities, cities_list
for city in cities_list:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&APPID={key}"
    urls.append(url)

df = pd.DataFrame()
for url in urls:
    res = requests.get(url)
    if res.status_code != 200:
        continue

    K2C = lambda k: k - 273.15
    data = json.loads(res.text)
    dict_temp = {
        '도시': [data['name']],
        '날씨': [data['weather'][0]['description']],
        '최저기온(섭씨)': [K2C(data['main']['temp_min'])],
        '최고기온(섭씨)': [K2C(data['main']['temp_max'])],
        '습도': [data['main']['humidity']],
        '기압': [data['main']['pressure']],
        '풍향': [data['wind']['deg']],
        '풍속': [data['wind']['speed']],
    }
    print(dict_temp)
    df_temp = pd.DataFrame(dict_temp)
    df = pd.concat([df, df_temp])

df = df.reset_index(drop=True)

{'도시': ['Tokusan-ri'], '날씨': ['clear sky'], '최저기온(섭씨)': [24.08000000000004], '최고기온(섭씨)': [24.08000000000004], '습도': [61], '기압': [1001], '풍향': [250], '풍속': [5.91]}
{'도시': ['Heunghae'], '날씨': ['few clouds'], '최저기온(섭씨)': [21.220000000000027], '최고기온(섭씨)': [21.220000000000027], '습도': [62], '기압': [999], '풍향': [267], '풍속': [6.33]}
{'도시': ['Reisui'], '날씨': ['clear sky'], '최저기온(섭씨)': [21.77000000000004], '최고기온(섭씨)': [21.77000000000004], '습도': [51], '기압': [1002], '풍향': [274], '풍속': [5.86]}
{'도시': ['Yeonil'], '날씨': ['clear sky'], '최저기온(섭씨)': [21.850000000000023], '최고기온(섭씨)': [21.850000000000023], '습도': [57], '기압': [999], '풍향': [268], '풍속': [6.93]}
{'도시': ['Neietsu'], '날씨': ['overcast clouds'], '최저기온(섭씨)': [13.060000000000002], '최고기온(섭씨)': [13.060000000000002], '습도': [95], '기압': [1000], '풍향': [255], '풍속': [2.28]}
{'도시': ['Yeongju'], '날씨': ['overcast clouds'], '최저기온(섭씨)': [13.450000000000045], '최고기온(섭씨)': [13.450000000000045], '습도': [83], '기압': [1000], '풍향': [272], '풍속': [4.03]}
{'도시': ['Yongin'], 

{'도시': ['Yongsan'], '날씨': ['clear sky'], '최저기온(섭씨)': [15.660000000000025], '최고기온(섭씨)': [19.900000000000034], '습도': [53], '기압': [1000], '풍향': [274], '풍속': [5.14]}
{'도시': ['Sinch’ŏn-dong'], '날씨': ['scattered clouds'], '최저기온(섭씨)': [15.670000000000016], '최고기온(섭씨)': [19.910000000000025], '습도': [57], '기압': [1000], '풍향': [278], '풍속': [4.48]}
{'도시': ['Sangok'], '날씨': ['clear sky'], '최저기온(섭씨)': [22.5], '최고기온(섭씨)': [22.5], '습도': [48], '기압': [1002], '풍향': [284], '풍속': [6.45]}
{'도시': ['Sang-ni'], '날씨': ['few clouds'], '최저기온(섭씨)': [20.970000000000027], '최고기온(섭씨)': [20.970000000000027], '습도': [60], '기압': [1000], '풍향': [270], '풍속': [2.06]}
{'도시': ['Sangju'], '날씨': ['scattered clouds'], '최저기온(섭씨)': [21.77000000000004], '최고기온(섭씨)': [21.77000000000004], '습도': [55], '기압': [1000], '풍향': [279], '풍속': [6.79]}
{'도시': ['Samcheok'], '날씨': ['scattered clouds'], '최저기온(섭씨)': [14.230000000000018], '최고기온(섭씨)': [14.230000000000018], '습도': [74], '기압': [999], '풍향': [289], '풍속': [2.76]}
{'도시': ['Shisen'], '날씨': ['clear

{'도시': ['Gumi'], '날씨': ['few clouds'], '최저기온(섭씨)': [21.75], '최고기온(섭씨)': [21.75], '습도': [49], '기압': [1000], '풍향': [279], '풍속': [6.53]}
{'도시': ['Goyang-si'], '날씨': ['clear sky'], '최저기온(섭씨)': [15.629999999999995], '최고기온(섭씨)': [19.870000000000005], '습도': [52], '기압': [999], '풍향': [290], '풍속': [5.33]}
{'도시': ['Goseong'], '날씨': ['clear sky'], '최저기온(섭씨)': [24.02000000000004], '최고기온(섭씨)': [24.02000000000004], '습도': [56], '기압': [1001], '풍향': [246], '풍속': [5.53]}
{'도시': ['Koryŏng'], '날씨': ['clear sky'], '최저기온(섭씨)': [21.910000000000025], '최고기온(섭씨)': [23.879999999999995], '습도': [42], '기압': [1000], '풍향': [282], '풍속': [4.49]}
{'도시': ['Gongju'], '날씨': ['clear sky'], '최저기온(섭씨)': [17.900000000000034], '최고기온(섭씨)': [17.900000000000034], '습도': [48], '기압': [1002], '풍향': [285], '풍속': [5.9]}
{'도시': ['Kyosai'], '날씨': ['few clouds'], '최저기온(섭씨)': [24.060000000000002], '최고기온(섭씨)': [24.060000000000002], '습도': [83], '기압': [1002], '풍향': [232], '풍속': [7.79]}
{'도시': ['Koyo'], '날씨': ['clear sky'], '최저기온(섭씨)': [21.29000

{'도시': ['Jeollanam-do'], '날씨': ['clear sky'], '최저기온(섭씨)': [21.32000000000005], '최고기온(섭씨)': [21.32000000000005], '습도': [43], '기압': [1003], '풍향': [297], '풍속': [6.09]}
{'도시': ['Jeollabuk-do'], '날씨': ['clear sky'], '최저기온(섭씨)': [18.460000000000036], '최고기온(섭씨)': [18.460000000000036], '습도': [59], '기압': [1003], '풍향': [286], '풍속': [5.61]}
{'도시': ['Jinju'], '날씨': ['clear sky'], '최저기온(섭씨)': [23.900000000000034], '최고기온(섭씨)': [23.900000000000034], '습도': [41], '기압': [1001], '풍향': [275], '풍속': [4.12]}
{'도시': ['Chinhae'], '날씨': ['clear sky'], '최저기온(섭씨)': [24.02000000000004], '최고기온(섭씨)': [24.02000000000004], '습도': [57], '기압': [1001], '풍향': [254], '풍속': [5.61]}
{'도시': ["Chinch'ŏn"], '날씨': ['clear sky'], '최저기온(섭씨)': [15.32000000000005], '최고기온(섭씨)': [19.560000000000002], '습도': [66], '기압': [1001], '풍향': [282], '풍속': [6.95]}
{'도시': ['Jinan-gun'], '날씨': ['clear sky'], '최저기온(섭씨)': [19.510000000000048], '최고기온(섭씨)': [19.510000000000048], '습도': [55], '기압': [1002], '풍향': [281], '풍속': [5.64]}
{'도시': ['Shitsukoku']

{'도시': ['Wonteo'], '날씨': ['broken clouds'], '최저기온(섭씨)': [14.970000000000027], '최고기온(섭씨)': [19.210000000000036], '습도': [77], '기압': [1000], '풍향': [275], '풍속': [4.84]}
{'도시': ['Dongjinwon'], '날씨': ['broken clouds'], '최저기온(섭씨)': [14.920000000000016], '최고기온(섭씨)': [19.160000000000025], '습도': [77], '기압': [1000], '풍향': [277], '풍속': [5.08]}
{'도시': ['Haejeong'], '날씨': ['overcast clouds'], '최저기온(섭씨)': [15.710000000000036], '최고기온(섭씨)': [19.950000000000045], '습도': [84], '기압': [1001], '풍향': [250], '풍속': [5.66]}
{'도시': ['Sangdeok'], '날씨': ['broken clouds'], '최저기온(섭씨)': [14.800000000000011], '최고기온(섭씨)': [19.04000000000002], '습도': [85], '기압': [1000], '풍향': [277], '풍속': [5.49]}
{'도시': ['Samga-dong'], '날씨': ['broken clouds'], '최저기온(섭씨)': [14.730000000000018], '최고기온(섭씨)': [18.970000000000027], '습도': [77], '기압': [1000], '풍향': [277], '풍속': [5.3]}
{'도시': ['Seryui-dong'], '날씨': ['overcast clouds'], '최저기온(섭씨)': [15.600000000000023], '최고기온(섭씨)': [19.840000000000032], '습도': [94], '기압': [1001], '풍향': [275], '풍속':

# 5. Check Dataframe

In [62]:
df

,도시,날씨,최저기온(섭씨),최고기온(섭씨),습도,기압,풍향,풍속
0,Tokusan-ri,clear sky,24.08,24.08,61,1001,250,5.91
1,Heunghae,few clouds,21.22,21.22,62,999,267,6.33
2,Reisui,clear sky,21.77,21.77,51,1002,274,5.86
3,Yeonil,clear sky,21.85,21.85,57,999,268,6.93
4,Neietsu,overcast clouds,13.06,13.06,95,1000,255,2.28
...,...,...,...,...,...,...,...,...
237,Chidong-gol,broken clouds,23.84,23.84,46,1001,281,5.84
238,Ugok,broken clouds,23.93,23.93,46,1001,281,5.82
239,Changnyeong,broken clouds,23.67,23.67,46,1000,278,5.75
240,Changsal-li,overcast clouds,14.47,14.47,93,999,309,1.71


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   도시        242 non-null    object 
 1   날씨        242 non-null    object 
 2   최저기온(섭씨)  242 non-null    float64
 3   최고기온(섭씨)  242 non-null    float64
 4   습도        242 non-null    int64  
 5   기압        242 non-null    int64  
 6   풍향        242 non-null    int64  
 7   풍속        242 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 15.2+ KB
